# Contents
1) create a search table with key=term and value paragragh in raw LaTeXML output

2) a term-reference is {term, art_addr, parag_index, p_tag, tfidf}

3) serialize the list of term-references

TODO:
- Some `<para>` tags don't have a `<p>` inside so I just used recutext. *ADD* the missing p tag

In [1]:
from lxml import etree
import pickle
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import glob
from tqdm import tqdm
from sklearn import linear_model
import tarfile
from tqdm import tqdm
from marshmallow import Schema, fields, pprint
from random import random
from dataclasses import dataclass, field

import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, os.path.join(parentdir, 'embed'))
sys.path.insert(0, parentdir)

from clean_and_token_text import normalize_text, normalize_phrase
import parsing_xml as px


In [15]:
general_path = 'math20/2003_003'
argot_path = '/media/hd1/glossary/NN.v1/' + general_path + '.xml.gz'
prom_path = '/media/hd1/promath/' + general_path + '.tar.gz'
ns = {'latexml': 'http://dlmf.nist.gov/LaTeXML' }

argot = etree.parse(argot_path)
with tarfile.open(prom_path) as tar_fobj:
    art_lst = [k.get_info()['name'] for k in tar_fobj.getmembers()]

In [16]:
def get_para(art_addr, para, tar_path=prom_path, run_recutext=True):
    with tarfile.open(tar_path) as tar_fobj:
        xml_fobj = tar_fobj.extractfile(art_addr)
        art = etree.parse(xml_fobj)
        para_lst = art.findall('.//latexml:para', namespaces=ns)
        para_tag = para_lst[para]
        #p_tag = etree.tostring(
        #    para_tag.find('latexml:p', namespaces=ns)).decode('utf-8')
        if run_recutext:
            return px.recutext_xml(para_tag)
        else:
            return etree.tostring(p_tag).decode('utf-8')
get_para("1501_005/1501.02441/Tiling.xml", 20)

KeyError: "filename '1501_005/1501.02441/Tiling.xml' not found"

In [17]:
# Define a dictionary Key=term and Value=(article_address, parag_index)
#results = argot.xpath(".//dfndum[text()='isothermic']")
#results = argot.xpath(".//dfndum[contains(text(),'quan')]")
results = argot.xpath(".//dfndum")
term_pair_dict = defaultdict(list)
for r in results:
    parent = r.getparent()
    para_index = int(parent.get('index'))
    grand_parent = parent.getparent()
    gparent_name = grand_parent.get('name')
    term_pair_dict[normalize_phrase(r.text)].append((gparent_name, para_index))
    
term_para_dict = defaultdict(list)
with tarfile.open(prom_path) as tar_fobj:
    for term, pair_lst in tqdm(term_pair_dict.items()):
        for pair in pair_lst:
            xml_fobj = tar_fobj.extractfile(pair[0])
            art = etree.parse(xml_fobj)
            para_lst = art.findall('.//latexml:para', namespaces=ns)
            para_tag = para_lst[pair[1]]
            try:
                #p_tag = etree.tostring(
                #    para_tag.find('.//latexml:p', namespaces=ns)).decode('utf-8').strip()
                p_tag = etree.tostring(
                         para_tag).decode('utf-8').strip()
            except TypeError as e:
                p_tag = px.recutext_xml(para_tag)
                print(e)
            term_para_dict[term].append((*pair, p_tag))
    

100%|██████████| 575/575 [01:08<00:00,  8.37it/s]


In [5]:
# search for term containing some text and get the paragraphs (cleaned)
results = argot.xpath(".//dfndum[contains(text(),'quant')]")
with tarfile.open(prom_path) as tar_fobj:
    for r in results:
        print(etree.tostring(r))
        parent = r.getparent()
        para_index = int(parent.get('index'))
        grand_parent = parent.getparent()
        gparent_name = grand_parent.get('name')
        print(grand_parent.tag, gparent_name, grand_parent.get('num') )
        xml_fobj = tar_fobj.extractfile(gparent_name)
        #art = etree.parse(xml_fobj.extractfile())
        art = etree.parse(xml_fobj)
        para_lst = art.findall('.//latexml:para', namespaces=ns)
        print(px.recutext_xml(para_lst[para_index]))
        print('----------------------------')

b'<dfndum>Locally covariant quantum field</dfndum>'
article 1501_005/1501.02682/1501.02682.xml 131
 Locally covariant quantum field theory _citation_ describes QFT on a category of globally hyperbolic spacetimes _inline_math_. Fixing a spacetime dimension _inline_math_, objects of _inline_math_ are quadruples _inline_math_ where _inline_math_ is a smooth paracompact orientable nonempty _inline_math_-manifold with finitely many connected components, _inline_math_ is a smooth time-orientable metric of signature _inline_math_ on _inline_math_, _inline_math_ and _inline_math_ are choices of orientation and %****␣1501.02682.tex␣Line␣600␣**** time-orientation respectively, 1 The orientation (resp., time-orientation) is conveniently represented as a choice of one of the connected components of the nowhere-zero smooth _inline_math_-forms (resp., _inline_math_-timelike _inline_math_-forms) on _inline_math_. so that the spacetime _inline_math_ is globally hyperbolic. That is, _inline_math_ has n

In [19]:
@dataclass
class TermReference(Schema):
    term : str
    addr : str
    index : int
    p_tag : str
    tfidf : float = field(default_factory=random)
    
class TermRefSchema(Schema):
    term = fields.String()
    addr = fields.String()
    index = fields.Int()
    p_tag = fields.String()
    tfidf = fields.Float()

term_ref_lst=[]
for term, tup_lst in term_para_dict.items():
    for tup in tup_lst:
        term_ref_lst.append(TermReference(
            term= term,
               addr = tup[0],
               index = tup[1],
              p_tag = tup[2],
               tfidf = random()))

# Serialize all the data and save to json file
trs = TermRefSchema(many=True)
with open('/home/luis/rm_me/math_json/'+ general_path + '.json', 'w') as fobj:
    fobj.write(trs.dumps(term_ref_lst))